# Notebook 13.2: RAD-seq assembly

### Learning objectives: 

By the end of this notebook you will:

1. Learn to use the ipyrad software for RAD-seq assembly.
2. Be able to describe the difference between denovo and reference assemblies.
3. Be familiar with output RAD-seq files for downstream analyses. 


### Assigned readings:

1. Andrews, Kimberly R., Jeffrey M. Good, Michael R. Miller, Gordon Luikart, and Paul A. Hohenlohe. 2016. “Harnessing the Power of RADseq for Ecological and Evolutionary Genomics.” Nature Reviews Genetics 17 (2): 81. https://doi.org/10.1038/nrg.2015.28.



2. Eaton, Deren A. R., Andrew L. Hipp, Antonio González-Rodríguez, and Jeannine Cavender-Bares. 2015. “Historical Introgression among the American Live Oaks and the Comparative Nature of Tests for Introgression.” Evolution 69 (10): 2587–2601. https://doi.org/10.1111/evo.12758.



In [ ]:
import ipyrad as ip
import toyplot
import toytree

### Assembling RAD-seq data sets
In the last notebook you examined some RAD-seq data in fastq format. The goal of RAD-seq assembly is to take all of these short-read sequences from across the genome and to accurately construct orthologous loci across samples. This can be done in one of two ways: (1) by mapping to a reference genome; or (2) by clustering *de novo* based on sequence similarity. These approaches each have different advantages particularly when working across different evolutionary scales and in systems with or without a well assembled reference genome. 

The final results of an assembly of RAD-seq data can be *variant calls*, in which case you are interested in finding SNPs that vary among the individuals in your data set; or it can include assembled loci, in which case the results are loci constructed from the overlapping reads adjacent to each restriction recognition site in the genome. The latter is particularly useful for phylogenetics or some population genetic inference methods that require information about variant sites as well as invariants sites in the genome. 

### Standard pipelines (reference map)

The methods we've used previously for mapping shotgun sequence data to a reference genome (e.g., bwa) can similarly be used to map RAD-seq data to a reference. Afterall, RAD-seq is just Illumina short-read data that covers a subset of the genome. We'll cover variant calling methods for whole genome sequence data in the coming week. There are many options and ways to do this and it usually involves combining many different software tools. 

### Specialized pipelines (denovo or reference map)
A simpler approach that can be appealing is to use an assembly pipeline that is able to accomplish all of the tasks involved in assembly within a single program call. For example, this would include read mapping, filtering, variant calling, and formatting output files for various downstream analyses, or providing the analysis functions built-in. A number of tools have been developed for this purpose that specialize in RAD-seq data. These tools usually offer methods for both reference-mapped assembly and *de novo* assembly of RAD-seq data sets. 

### ipyrad toolkit 

Here we'll use the program `ipyrad` for assembling and analyzing RAD-seq data sets. This is a tool written in Python that is designed to integrate well with Jupyter Notebooks for creating reproducible workflows. Feel free to explore the [ipyrad documentation](https://ipyrad.readthedocs.io) for details. In a similar manner to the `canu` software we learned about recently, `ipyrad` aims to be an all-in-one pipeline for assembling RAD-seq data starting from the raw sequence data to the final assembled data files. It can be called as a command line tool from a terminal, or be used within Python as a object-oriented package. We'll use the Python API version. 


### ipyrad Python API
The ipyrad API is centered around an object called an `Assembly`. This stores the parameters that you can set and which tell the program how to run, and it has functions that can be called to execute the assembly of the data. Below we start an assembly by creating a new Assembly object and providing it a name ("test"). The `.params` attribute returns a printed view of the parameter options. 

In [ ]:
# create an named object to represent the Assembly of the data
data = ip.Assembly("test")

# print all of the parameters options for this assembly
data.params

### Set parameters
As described in the documentation there is a large number of parameters that can be modified, but few that are required to be set. Here we will set the required parameters but leave most of the other optional parameters at their default settings. We enter the path to the data files and tell `ipyrad` to perform a reference-based assembly. We also set the parameter `project_dir` to the name "simdata". This will create a new folder in our current directory called "simdata" and organize all of the files produced by the assembly into that folder. 
Finally we print the parameters again which you can see have been updated from before. 

In [ ]:
# update the values for several parameters
data.params.project_dir = "~/work/simdata"
data.params.raw_fastq_path = "/home/jovyan/ro-data/ipsimdata/rad_example_R1_.fastq.gz"
data.params.barcodes_path = "/home/jovyan/ro-data/ipsimdata/rad_example_barcodes.txt"
data.params.reference_sequence = "/home/jovyan/ro-data/ipsimdata/rad_example_genome.fa"
data.params.assembly_method = "reference"

# print the new parameters
data.params

### Fast simulated example assembly
Once the parameters are set on your assembly object you can use the `.run()` command to run the steps of the assembly process. This will automatically distribute the work in parallel across multiple processors. Below we tell the program to run all seven steps of the assembly. This data set is quite small, only 500 loci across 12 samples. It can complete the assembly using four processors in about a minute. Here you can simply watch the entire assembly run. Next we'll break it down to examine what is happening in each step of the assembly. 

The assembly process in ipyrad is broken into seven steps, as shown below. This atomized process is useful as you can examine how changing different parameters of the assembly process influences the results. For example you may try one assembly with a set of parameters, and then another that branches from the first assembly at step 5 and uses a different set of parameters for the remaining few steps. We'll see examples of this later. 

![https://ipyrad.readthedocs.io/_images/steps.png](https://ipyrad.readthedocs.io/_images/steps.png)


The `force=True` argument is set here to tell the `.run()` command to overwrite an existing data assembly if one already exists with this name. This is required if you run this cell multiple times, otherwise it will not overwrite the existing assembly the second time you run it. The `auto=True` parameter automatically detects the available processors on the computer you are using and parallelizes the job. 

In [ ]:
# example: run the full assembly in one command
data.run("1234567", force=True, auto=True)

## Steps of the ipyrad assembly

### Step 1: demultiplexing/loading reads
Step 1 of the assembly has two possible functions. If the data are already demultiplexed then you can provide `ipyrad` with the fastq files for each individual in your data set and it will simply load in and count the number of reads for each sample. If your data are not yet demultiplexed then you must provide the *raw fastq file* with all of your sequence data and a *barcodes file* that maps sample names to their barcode sequences. That is the approach we are using here. We start by demultiplexing the sequences to samples. 

In [ ]:
# we set the raw fastq path earlier 
data.params.raw_fastq_path

In [ ]:
# we set the barcodes file earlier
data.params.barcodes_path

In the last notebook we examined the protocol for preparing a RAD-seq library which included ligating barcodes fragmented genome sequences. This information is one of the required inputs for demultiplexing the raw reads and assigning them to each individual sample. The file above simply lists in tab-delimited form a sample name and barcode sequence on each line. We can access the parsed form of this information as a dictionary as shown below by accessing the `.barcodes` attrbute of the Assembly object. During step 1 ipyrad will use this information. 

In [ ]:
# the barcode info can be viewed as a dictionary mapping names to barcodes
data.barcodes

In this case we are restarting the same assembly that we had previously completed. It has the same assembly name and project_dir. To overwrite the previous assembly we use the force=True flag.

In [ ]:
# run step 1 to demultiplex the data
data.run("1", force=True, auto=True)

In [ ]:
# examine the results of step 1 demultiplexing
data.stats

The same general framework can be followed for the remaining steps. Run the step, check the stats output, and perhaps modify parameters that affect that step to see how they change the results. The documentation has details on which parameters affect which steps of the assembly. 

A convenience of being able to access the statistics of the assembly in Python and as a DataFrame is that we can easily calculate further values from the results. For example, if we want to know the total number of reads across all samples we can calculate the sum. Similarly, we could calculate variance, or plot the values, etc. 

In [ ]:
# total reads across all samples
data.stats.reads_raw.sum()

### Step 2: Filtering reads
We've discussed previously the importance of filtering and trimming short reads before using them for a whole genome assembly. This is also true for RAD-seq data sets. Step 2 in ipyrad performs this step. Because we are currently working with a simulated data set there is no adapter contamination and the reads are all high quality. But we will see later in empirical examples that this step can be very important. If we wanted to set a higher stringency for filtering we can increase the `filter_adapters` parameter to a higher value (options are 0, 1, 2, or 3). Further details are in the documentation. 

In [ ]:
# set the filtering parameter to a more stringent threshold
data.params.filter_adapters = 2

# continue this assembly from step 1 to step 2
data.run("2", force=True, auto=True)

In [ ]:
# view the stats summary
data.stats

In [ ]:
# there are also more detailed stats summaries avaiable for each step
data.stats_dfs.s2

### Step 3: Cluster or map reads
Depending on whether the `assembly_method` parameter is set to *denovo* or *reference* this step will perform a slightly different task. In denovo mode it uses a clustering algorithm to identify reads within each sample that are sequenced copies from the same locus. If in reference mode then the same thing is accomplished by mapping reads to a reference genome. In either case, we expect that many reads are assigned to each locus in the dataset so that we have high coverage for making accurate base calls later. 

In [ ]:
# continue this assembly from step 2 to step 3
data.run("3", force=True, auto=True)

In [ ]:
# the detailed statistics for step 3 read mapping
data.stats_dfs.s3

### Steps 4-5: Consensus haplotype calling 
Once reads are mapped to each locus ipyrad then aims to make consensus haplotype calls within each individual. With high coverage data we expect to recover both alleles for diploid organisms and to correct for sequencing errors. Step 4 uses the read coverage information to first jointly estimate the error rate and heterozygosity. Then step 5 uses these values to make base calls and infer haplotypes. 

In [ ]:
# run steps 4 and 5
data.run("45", force=True, auto=True)

In [ ]:
# look at summarized stats so far
data.stats

### Step 6: Across sample orthology
Up to this point processing has occurred in each sample individually. Now in step 6 we either use clustering again to identify orthology among consensus reads from different individuals in *denovo* assemblies, or we use the reference mapped locations to build loci of orthologous consensus sequences across samples for *reference-based* assemblies. 

For closely related organisms the reference-based assemblies will usually give the best results, and will run much faster. However, if you do not have a closely related genome to your study clade, or if the organisms in your study span a very wide phylogenetic scale (e.g., many tens of millions of years of divergence) then using a single reference genome to align their reads may end up throwing away much of your data since the organisms may have very divergent genomes. In that case the *denovo* assembly method can yield a lot more data for phylogenetic scale studies. 

Here we are continuing with a reference-based assembly. 

In [ ]:
# build loci from orthologous consensus reads
data.run("6", force=True, auto=True)

<div class="alert alert-success">
    <b>[6] Question:</b> 
    What are advantages or disadvantages to using de novo versus reference-mapped RAD-seq loci? Answer in markdown below. 
</div>

In [ ]:
# answer here

### Step 7: Apply filters and format output files
This is typically the most important step of an ipyrad assembly. At step 7 you can apply different filters to your data and produce output files for downsteam analyes that include or exclude certain samples or loci that can make the data more suitable for different types of analyses. 

This is especially relevant to the issue of *missing data* in RAD-seq data sets. Many factors can contribute to why and how data is missing in RAD-seq data sets, and many papers have been written on the subject. Allelic dropout can lead to missing data when mutations occur in restriction recognition sites. Low sequencing coverage can lead to some regions not being sequenced. Uneven amplification of the library can lead to some fragments being sequenced to very high depth and others to very low depth. Whatever the cause, there is often missing data in RAD-seq data sets and the final step of an assembly should aim to minimize the missing data depending on your expected downstream analyses. 

For example, many studies have shown that missing data has little impact on phylogenetic inference methods, whereas it can hugely influence principle components analysis, or population structure analyses. We will explore missing data more in the next notebook. 

In [ ]:
# only keep loci that have data for N sample or more
data.params.min_samples_locus = 10

# run final assembly step
data.run("7", force=True, auto=True)

### The output files
All of the files of this assembly are organized into the `project_dir` that we set as a parameter. Open a terminal from the jupyter dashboard page (select New then Terminal from that page) and navigate to this folder. Use the `less` command to look at the contents of each of the output files. Be sure to look at the stats output file labeled with the `_stat.txt` suffix. This file summarizes the stats output of the assembly. Because this is a simple assembly of simulated data the stats output is not too interesting though. We'll see another example from an empirical data set next time. 

In [ ]:
# the outfile paths are also accessible through the Assembly object
data.outfiles

### The .loci file
This is a human readable format similar to a fasta format. It highlights the SNPs in the data and is easy to parse. It is useful to look at your results files to make sure that there are no apparent errors in your data or assembly. For example, a quick look will reveal if you mislabeled a taxon, or if one sample completely failed and produced no data. 

<div class="alert alert-success">
    <b>[7] Question:</b> 
    Look at the output file produced by step 7 of ipyrad that has the .loci suffix in the directory shown above. How many SNPs are in the first locus?  Answer in Markdown below.
</div>

In [ ]:
# answer here

### The VCF file
The VCF file (variant call format file) differs from the .loci file above in that it only shows the variant sites, and excludes the invariant sites. This is typical for VCF files. However, it contains additional information the other file formats lack in terms of the depths of coverage used during base calling. You can see in this file the numbers of A, T, C, and Gs that were present when a base was called homozygous or heterozygous in one or more samples at a variable site. This is useful because many downstream analysis tools can apply further filtering on the data in this file based on coverage and certainty in base calls. 

### The phylip file
The phylip formatted file (.phy) contains the sequence data concatenated into one giant sequence alignment. This file can be used in some phylogenetic inference software. As an example, we can stick it into an analysis tool of ipyrad below to infer a phylogenetic tree and then plot it. 

In [ ]:
import ipyrad.analysis as ipa
import toytree

# infer a ML tree
rax = ipa.raxml(data=data.outfiles.phy, N=10)
rax.run(block=True, force=True)

# plot the tree
tre = toytree.tree(rax.trees.bipartitions)
tre.root(wildcard="3").draw();

<div class="alert alert-success">
    <b>[8] Question:</b> 
    From the inferred tree above, if we assume each of the tips is a different species, which species do you think the reference genome is most closely related to?
</div>

In [ ]:
# answer here

## Phylogenetics II

You have now successfully assembled a RAD-seq comparative genomic data set and inferred a phylogeny from the resulting data. Over the next few notebooks to be assigned you will continue to apply RAD-seq methods to assemble a larger empirical data set and to apply phylogenetic and population genetic inference methods. For this we will focus on the Oak data set from the Eaton et al. 2015 paper that was assigned as a reading for this session. I understand that this paper is fairly complex and some students might struggle to follow all of the methods, especially if this topic is not your background. We will dissect the paper in class in detail in terms of the data assembly and the methods that are employed. For now if it feels over your head then please just at least skim the paper which should be sufficient to answer the questions below. 

<div class="alert alert-success">
    <b>[9] Question:</b> 
    See Table 1 in the Eaton et al. paper which shows the number of RAD-seq loci that were included in de novo assembled data sets that were assembled with different parameter settings. The Allmin4 and Allmin20 assemblies differ greatly in the total number of loci, the number of informative sites, and the missing data. Why do you think 
</div>

In [ ]:
# answer here

<div class="alert alert-success">
    <b>[10] Question:</b> 
    What does the term introgression mean? What do the authors argue are "two important considerations" to take when testing for introgression with genomic data? 
</div>

In [ ]:
# answer here